In [ ]:
#1 - k-means clustering 

In [ ]:
#2 - supervised learning because it iterates each set and compares the out of the algorithm used.
#It is then adjusts parameters to make the output closer/equal to the desired output

In [ ]:
#3 - a test set is a set of data that is used on the machine learning program after it has been trained on the initial data set.
#It can test to see how well the program was trained and to see if it is accurate on the new data set

In [1]:
import boto3 
import pandas as pd

#defining bucket
s3 = boto3.resource('s3')
bucket_name = 'data445-ashlyn-bucket' 
bucket = s3.Bucket(bucket_name)

#defining csv file
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body') 

#reading csv file
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [2]:
#removing NA
heart = heart.dropna()

In [3]:
#defining the input and target variables
X = heart[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate']]
Y = heart['TenYearCHD']

from sklearn.model_selection import train_test_split

#splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20)

In [4]:
from sklearn.linear_model import LogisticRegression

#building the logistic model
logit_md = LogisticRegression().fit(X_train, Y_train)

#prodicting the likelihood of TenYearCHD
preds = logit_md.predict_proba(X_test)[:,1]
preds

array([0.23248542, 0.06010488, 0.05623258, 0.0726718 , 0.13344136,
       0.17424493, 0.06219911, 0.06175032, 0.08534994, 0.06637636,
       0.03772718, 0.08878461, 0.1771488 , 0.25807532, 0.05142898,
       0.47635942, 0.09672054, 0.08867601, 0.33155829, 0.03943462,
       0.09284312, 0.06319558, 0.06755237, 0.14709747, 0.22275055,
       0.07304017, 0.22843373, 0.05831506, 0.22595083, 0.18712033,
       0.10113409, 0.08481668, 0.13910137, 0.09153244, 0.18038289,
       0.06840565, 0.27383171, 0.14535535, 0.28869367, 0.17282256,
       0.33460746, 0.09792529, 0.07730734, 0.20563164, 0.26719407,
       0.06702048, 0.12033488, 0.07340271, 0.05950565, 0.19762299,
       0.28818576, 0.0567196 , 0.10987501, 0.20080953, 0.0843622 ,
       0.07678833, 0.09172086, 0.14923623, 0.0788453 , 0.11707525,
       0.07669016, 0.0700477 , 0.24521735, 0.31459954, 0.07783054,
       0.2049402 , 0.09463   , 0.04069152, 0.05287752, 0.17717239,
       0.22919423, 0.23871088, 0.21925259, 0.16434196, 0.26333

In [5]:
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix

preds = np.where(preds < 0.25, 0, 1)

#creating the confusion matrix
confusion_matrix(Y_test, preds)

#extracting the accuracy score
accuracy_score(Y_test, preds)

0.7814207650273224

In [ ]:
#defining the input and target variables
X = heart[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate']]
Y = heart['TenYearCHD']

from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LinearRegression

#defining the list to store result
md1_results = list()

cv = LeaveOneOut()

for train_ix, test_ix in cv.split(X):
    
    #splitting the data
    X_train, X_test = X.loc[train_ix], X.loc[test_ix]
    Y_train, Y_test = Y.loc[train_ix], Y.loc[test_ix]
    
    #building the linear model
    lm_md = LinearRegression().fit(X_train, Y_train)
    
    #predicting on the test set
    pred = lm_md.predict(X_test)
    
    #storing the results
    md1_results.append(pred[100])
    
#computing the mse
mse_md1 = np.mean(np.power(md1_results - Y, 100))
mse_md1